In [ ]:
from keras.layers import Input, Activation, Conv2D, Flatten, Dense, MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from IPython.display import Image
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from imutils import face_utils
import dlib
x_train=(np.load('./x_train.npy'))
y_train=(np.load('./y_train.npy'))
x_val=(np.load('./x_val.npy'))
y_val=(np.load('./y_val.npy'))


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    x=x_train, y=y_train,
    batch_size=32,
    shuffle=True
)

val_generator = val_datagen.flow(
    x=x_val, y=y_val,
    batch_size=32,
    shuffle=False
)

In [ ]:
inputs = Input(shape=(26, 34, 1))

net = Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
net = MaxPooling2D(pool_size=2)(net)

net = Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')(net)
net = MaxPooling2D(pool_size=2)(net)

net = Flatten()(net)

net = Dense(512)(net)
net = Activation('relu')(net)
net = Dense(1)(net)
outputs = Activation('sigmoid')(net)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26, 34, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 34, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 17, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 17, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 8, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 4, 128)         0     

In [ ]:
model.fit_generator(
    train_generator, epochs=50, validation_data=val_generator,)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
81/81 [==============================] - 7s 70ms/step - loss: 0.5027 - acc: 0.7309 - val_loss: 0.2907 - val_acc: 0.8854
Epoch 2/50
81/81 [==============================] - 5s 67ms/step - loss: 0.2953 - acc: 0.8739 - val_loss: 0.1672 - val_acc: 0.9167
Epoch 3/50
81/81 [==============================] - 6s 68ms/step - loss: 0.1739 - acc: 0.9362 - val_loss: 0.1417 - val_acc: 0.9375
Epoch 4/50
81/81 [==============================] - 5s 67ms/step - loss: 0.1510 - acc: 0.9428 - val_loss: 0.1510 - val_acc: 0.9549
Epoch 5/50
81/81 [==============================] - 5s 67ms/step - loss: 0.1361 - acc: 0.9540 - val_loss: 0.0731 - val_acc: 0.9896
Epoch 6/50
81/81 [==============================] - 5s 68ms/step - loss: 0.1316 - acc: 0.9536 - val_loss: 0.0654 - val_acc: 0.9757
Epoch 7/50
81/81 [==============================] - 5s 68ms/step - loss: 0.1146 - acc: 0.9590 - val_loss: 0.0706 - val_acc: 0.9722
Epoch 8/50
81/81 [==============================] - 5s 67ms/step - loss: 0.0863 - a

In [ ]:
class CustomDataSet(Dataset):
  def __init__(self, x, y, transform=None):
    self.transform = transform
    self.X=x
    self.Y=y

  def __len__(self):
        return len(self.X)

  def __getitem__(self, idx):
        data = self.X[idx]
        data=Image.fromarray(np.squeeze(data, axis=2))
        data=data.convert('L')
        label = self.Y[idx]

        if self.transform:
            img_as_tensor = self.transform(data)

        return (img_as_tensor,label)

train_dataset=CustomDataSet(x_train,y_train,data_transform)
train_loader = DataLoader(dataset=train_dataset,batch_size=64,shuffle=True)
# test_dataset=CustomDataSet(x_val,y_val)
# test_loader = iter(DataLoader([x_val,y_val], batch_size=32))




  

In [ ]:
h,w=x_train.shape[2:4]
x_train.shape

(2586, 26, 34, 1)

In [ ]:
class Net(torch.nn.Module):   
    def __init__(self,input_dim):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 32, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(32),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )
        num_features_before_fcnn = functools.reduce(operator.mul, list(self.cnn_layers(torch.rand(1, (*input_dim))).shape))
        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x